In [296]:
import pandas as pd
import plotly.express as px
import numpy as np
import scipy
import geopandas as gpd
import matplotlib.pyplot as plt
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim

World Happiness Report from 2015-2019.
Kaggle File: https://www.kaggle.com/datasets/unsdsn/world-happiness?select=2015.csv

World map, region definitions: https://ourworldindata.org/world-region-map-definitions

In [297]:
#read in all the files
happy_2015 = pd.read_csv('2015.csv')
happy_2016 = pd.read_csv('2016.csv')
happy_2017 = pd.read_csv('2017.csv')
happy_2018 = pd.read_csv('2018.csv')
happy_2019 = pd.read_csv('2019.csv')
country_region = pd.read_csv('continents-according-to-our-world-in-data.csv')

In [298]:
#Add "Year" column
file_list = [happy_2015,happy_2016,happy_2017,happy_2018,happy_2019]
i = 2015
for data in file_list:
    data['Year'] = i
    i += 1

In [299]:
#Rename columns
happy_2015 = happy_2015.rename(columns={'Happiness Rank':'Rank','Happiness Score':'Happy_Score','Economy (GDP per Capita)':'Economy',
                                        'Health (Life Expectancy)':'Life_Expect','Trust (Government Corruption)':'Gov_Trust',
                                        'Dystopia Residual':'Dys_Res'})

happy_2016 = happy_2016.rename(columns={'Happiness Rank':'Rank','Happiness Score':'Happy_Score','Economy (GDP per Capita)':'Economy',
                                        'Health (Life Expectancy)':'Life_Expect','Trust (Government Corruption)':'Gov_Trust',
                                        'Dystopia Residual':'Dys_Res'})

happy_2017 = happy_2017.rename(columns={'Happiness.Rank':'Rank','Happiness.Score':'Happy_Score','Economy..GDP.per.Capita.':'Economy',
                                        'Health..Life.Expectancy.':'Life_Expect','Trust..Government.Corruption.':'Gov_Trust',
                                        'Dystopia.Residual':'Dys_Res'})

happy_2018 = happy_2018.rename(columns={'Overall rank':'Rank','Country or region':'Country','Score':'Happy_Score', 
                                        'GDP per capita':'Economy','Social support':'Family','Healthy life expectancy':'Life_Expect', 
                                        'Freedom to make life choices':'Freedom','Perceptions of corruption': 'Gov_Trust'})

happy_2019 = happy_2019.rename(columns={'Overall rank':'Rank','Country or region':'Country','Score':'Happy_Score', 
                                        'GDP per capita':'Economy','Social support':'Family','Healthy life expectancy':'Life_Expect',
                                        'Freedom to make life choices':'Freedom','Perceptions of corruption':'Gov_Trust'})

country_region = country_region.rename(columns={'Entity':'Country'})

In [300]:
#stack all dataframes into one, drop the columns that are not needed
stacked = pd.concat([happy_2015,happy_2016,happy_2017,happy_2018,happy_2019], ignore_index=True)
happy_data = stacked.drop(['Standard Error','Dys_Res','Lower Confidence Interval','Upper Confidence Interval','Whisker.high','Whisker.low','Region'],axis='columns')
country_region = country_region.drop(['Code','Year'], axis='columns')

In [301]:
def merge_region(df):
    df = pd.merge(df, country_region, how='left')
    return df

happy_data = pd.merge(happy_data, country_region, how='left')
happy_data.pipe(merge_region)

Country  Rank  Happy_Score  Economy   Family  \
0                 Switzerland     1        7.587  1.39651  1.34951   
1                     Iceland     2        7.561  1.30232  1.40223   
2                     Denmark     3        7.527  1.32548  1.36058   
3                      Norway     4        7.522  1.45900  1.33095   
4                      Canada     5        7.427  1.32629  1.32261   
..                        ...   ...          ...      ...      ...   
777                    Rwanda   152        3.334  0.35900  0.71100   
778                  Tanzania   153        3.231  0.47600  0.88500   
779               Afghanistan   154        3.203  0.35000  0.51700   
780  Central African Republic   155        3.083  0.02600  0.00000   
781               South Sudan   156        2.853  0.30600  0.57500   

     Life_Expect  Freedom  Gov_Trust  Generosity  Year      Continent  
0        0.94143  0.66557    0.41978     0.29678  2015         Europe  
1        0.94784  0.62877    0.14145     0.43630  2015         Europe  
2        0.87464  0.64938    0.48357     0.34139  2015         Europe  
3        0.88521  0.66973    0.36503     0.34699  2015         Europe  
4        0.90563  0.63297    0.32957     0.45811  2015  North America  
..           ...      ...        ...         ...   ...            ...  
777      0.61400  0.55500    0.41100     0.21700  2019         Africa  
778      0.49900  0.41700    0.14700     0.27600  2019         Africa  
779      0.36100  0.00000    0.02500     0.15800  2019           Asia  
780      0.10500  0.22500    0.03500     0.23500  2019         Africa  
781      0.29500  0.01000    0.09100     0.20200  2019         Africa  

[782 rows x 11 columns]

In [302]:
#Check the columns that have missing values
for col in happy_data.columns:
    print(f'no values for : {col}')
    print(happy_data[happy_data[col].isna()])
    print(' ')

print(happy_data[happy_data['Country']=='United Arab Emirates'])

#no need to put in the missing continents since it's irrelevant for this project

no values for : Country
Empty DataFrame
Columns: [Country, Rank, Happy_Score, Economy, Family, Life_Expect, Freedom, Gov_Trust, Generosity, Year, Continent]
Index: []
 
no values for : Rank
Empty DataFrame
Columns: [Country, Rank, Happy_Score, Economy, Family, Life_Expect, Freedom, Gov_Trust, Generosity, Year, Continent]
Index: []
 
no values for : Happy_Score
Empty DataFrame
Columns: [Country, Rank, Happy_Score, Economy, Family, Life_Expect, Freedom, Gov_Trust, Generosity, Year, Continent]
Index: []
 
no values for : Economy
Empty DataFrame
Columns: [Country, Rank, Happy_Score, Economy, Family, Life_Expect, Freedom, Gov_Trust, Generosity, Year, Continent]
Index: []
 
no values for : Family
Empty DataFrame
Columns: [Country, Rank, Happy_Score, Economy, Family, Life_Expect, Freedom, Gov_Trust, Generosity, Year, Continent]
Index: []
 
no values for : Life_Expect
Empty DataFrame
Columns: [Country, Rank, Happy_Score, Economy, Family, Life_Expect, Freedom, Gov_Trust, Generosity, Year, Conti

In [303]:
#impute the missing value for the UAE's trust number by filling it in with the average for all five years
def fill_nan_trust(df):
    gov_trust_avg = df.groupby('Country')['Gov_Trust'].mean().reset_index()
    uae = gov_trust_avg[gov_trust_avg['Country'] == 'United Arab Emirates'].reset_index()
    avg_num = round(uae.at[0, 'Gov_Trust'], 7)
    df['Gov_Trust'].fillna(avg_num, inplace = True)

    return df

happy_data.pipe(fill_nan_trust)

C:\Users\JINBX22\AppData\Local\Temp\ipykernel_14808\1413906187.py:6: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





Country  Rank  Happy_Score  Economy   Family  \
0                 Switzerland     1        7.587  1.39651  1.34951   
1                     Iceland     2        7.561  1.30232  1.40223   
2                     Denmark     3        7.527  1.32548  1.36058   
3                      Norway     4        7.522  1.45900  1.33095   
4                      Canada     5        7.427  1.32629  1.32261   
..                        ...   ...          ...      ...      ...   
777                    Rwanda   152        3.334  0.35900  0.71100   
778                  Tanzania   153        3.231  0.47600  0.88500   
779               Afghanistan   154        3.203  0.35000  0.51700   
780  Central African Republic   155        3.083  0.02600  0.00000   
781               South Sudan   156        2.853  0.30600  0.57500   

     Life_Expect  Freedom  Gov_Trust  Generosity  Year      Continent  
0        0.94143  0.66557    0.41978     0.29678  2015         Europe  
1        0.94784  0.62877    0.14145     0.43630  2015         Europe  
2        0.87464  0.64938    0.48357     0.34139  2015         Europe  
3        0.88521  0.66973    0.36503     0.34699  2015         Europe  
4        0.90563  0.63297    0.32957     0.45811  2015  North America  
..           ...      ...        ...         ...   ...            ...  
777      0.61400  0.55500    0.41100     0.21700  2019         Africa  
778      0.49900  0.41700    0.14700     0.27600  2019         Africa  
779      0.36100  0.00000    0.02500     0.15800  2019           Asia  
780      0.10500  0.22500    0.03500     0.23500  2019         Africa  
781      0.29500  0.01000    0.09100     0.20200  2019         Africa  

[782 rows x 11 columns]

Question 1a: Does a better economy mean higher happiness score?

Question 1b: How does the happiness score change for the top ten economies throughout the years.

In [306]:
def happy_economy(df):
    fig = px.scatter(happy_data, x='Economy', y='Happy_Score', title='Economy vs. Happiness Score').show()
    top_10_econ_2015 = happy_2015.nlargest(10, 'Economy')
    
happy_economy(happy_data)


                  Country  Rank  Happy_Score   Economy    Family  Life_Expect  \
489  United Arab Emirates    20        6.774  2.096000  0.776000     0.670000   
349                 Qatar    35        6.375  1.870766  1.274297     0.710098   
193                 Qatar    36        6.375  1.824270  0.879640     0.717230   
332            Luxembourg    18        6.863  1.741944  1.457584     0.845089   
177            Luxembourg    20        6.871  1.697520  1.039990     0.845420   
340             Singapore    26        6.572  1.692278  1.353814     0.949492   
27                  Qatar    28        6.611  1.690420  1.078600     0.797330   
654                 Qatar    29        6.374  1.684000  1.313000     0.871000   
501                 Qatar    32        6.374  1.649000  1.303000     0.748000   
179             Singapore    22        6.739  1.645550  0.867580     0.947190   

      Freedom  Gov_Trust  Generosity  Year Continent  
489  0.284000   0.311982    0.186000  2018      Asia 